In [ ]:
%matplotlib qt
import os
import os.path as op
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs, corrmap
from mne.minimum_norm import make_inverse_operator, apply_inverse

In [ ]:
sample_data_folder = op.expanduser("~/data/pic-name-data-bids/")
sample_data_raw_file = op.join(sample_data_folder, 'MEG/sub-08/ses-01/meg/sub-08_ses-01_task-picturenaming_run-01_meg.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file)
# raw.crop(tmin=120, tmax=180)
raw.crop(tmin=120, tmax=None)

In [ ]:
raw.drop_channels(['ECG062'])

In [ ]:
# regexp = r'(MEG[12][45][123]1)'
# artifact_picks = mne.pick_channels_regexp(raw.ch_names, regexp=regexp)
# raw.plot(order=artifact_picks, n_channels=len(artifact_picks), show_scrollbars=False)

In [ ]:
eog_evoked = create_eog_epochs(raw).average()
eog_evoked.apply_baseline(baseline=(None, -0.2))
eog_evoked.plot_joint()

In [ ]:
filt_raw = raw.copy()
filt_raw.load_data().filter(l_freq=1, h_freq=None)

In [ ]:
ica = ICA(n_components=15, random_state=97)
ica.fit(filt_raw)

In [ ]:
raw.load_data()
ica.plot_sources(raw, show_scrollbars=False)

In [ ]:
# ica.plot_components()

In [ ]:
# ica.plot_overlay(raw, exclude=[0], picks='mag')

In [ ]:
# ica.plot_properties(raw, picks=[0, 6])

In [ ]:
ica.exclude = [0]

In [ ]:
# raw = mne.io.read_raw_fif(sample_data_raw_file)
# raw.crop(tmin=120, tmax=None)

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', min_duration=0.001, shortest_event=1)
delay = int(round(0.056 * raw.info['sfreq']))
events[:, 0] = events[:, 0] + delay

In [ ]:
event_id = None
tmin = -0.5
tmax = 1.0
picks = mne.pick_types(raw.info, meg=True, eog=True, ecg=True, stim=False, exclude='bads')
baseline = (None, 0)
# reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)

In [ ]:
raw.load_data()
raw.filter(1, 20, picks=picks, filter_length='auto', n_jobs=1,
          method='fir', iir_params=None, phase='zero', fir_window='hamming',
          fir_design='firwin', skip_by_annotation=('edge', 'bad_acq_skip'),
          pad='reflect_limited', verbose=True)

In [ ]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                   baseline=baseline, reject=None, preload=True)

In [ ]:
ica.apply(epochs);

In [ ]:
noise_cov = mne.compute_covariance(epochs, tmax=0, method=['shrunk', 'empirical'])

In [ ]:
fig_cov, fig_spectra = mne.viz.plot_cov(noise_cov, raw.info)

In [ ]:
evoked = epochs.average().pick_types(meg=True)
evoked.plot(time_unit='s')
evoked.plot_topomap(times=np.linspace(0.05, 0.15, 5), ch_type='mag', time_unit='s')

In [ ]:
evoked.plot_white(noise_cov, time_unit='s')

In [ ]:
del epochs, raw

In [ ]:
subjects_dir = op.join(sample_data_folder, 'MRI')

In [ ]:
src = mne.setup_source_space('sub-08', spacing='oct6', subjects_dir=subjects_dir,
                            add_dist=False)
model = mne.make_bem_model(subject='sub-08', ico=4, conductivity=(0.33,),
                      subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [ ]:
trans_fname = op.join(sample_data_folder, 'MEG/sub-08/ses-01/meg/sub-08-trans.fif')

In [ ]:
fwd = mne.make_forward_solution(evoked.info, trans=trans_fname,
            src=src, bem=bem, meg=True, eeg=False, mindist=5.0, n_jobs=1)

fwd = mne.convert_forward_solution(fwd, surf_ori=True)

In [ ]:
info = evoked.info
inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8)

In [ ]:
del fwd

In [ ]:
method = "dSPM"
snr = 3
lambda2 = 1 / snr ** 2
stc, residual = apply_inverse(evoked, inverse_operator, lambda2, method=method, pick_ori=None,
                             return_residual=True, verbose=True)

In [ ]:
plt.figure()
plt.plot(1e3 * stc.times, stc.data[::100, :].T)
plt.xlabel('Time (ms)')
plt.ylabel('%s value' % method)
plt.show()

In [ ]:
reconst_raw = raw.copy()
ica.apply(reconst_raw)

In [ ]:
raw.plot(order=artifact_picks, n_channels=len(artifact_picks),
        show_scrollbars=False)
reconst_raw.plot(order=artifact_picks, n_channels=len(artifact_picks),
        show_scrollbars=False)

In [ ]:
del reconst_raw

In [ ]:
ica.exclude = []
eog_indices, eog_scores = ica.find_bads_eog(raw)
ica.exclude = eog_indices

In [ ]:
ica.plot_scores(eog_scores)

In [ ]:
ica.plot_properties(raw, picks=eog_indices)

In [ ]:
ica.plot_sources(raw, show_scrollbars=False)

In [ ]:
ica.plot_sources(eog_evoked)

In [ ]:
ica.exclude = []
ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='correlation', threshold='auto')
ica.exclude = ecg_indices

In [ ]:
ica.plot_scores(ecg_scores)

In [ ]:
ecg_indices

In [ ]:
ica.plot_properties(raw, picks=ecg_indices)

In [ ]:
ica.plot_sources(raw, show_scrollbars=False)

In [ ]:
new_ica = ICA(n_components=30, random_state=97)
new_ica.fit(filt_raw)

In [ ]:
ecg_indices, ecg_scores = new_ica.find_bads_ecg(raw, method='correlation', threshold='auto')
new_ica.exclude = ecg_indices

In [ ]:
ecg_indices

In [ ]:
new_ica.plot_scores(ecg_scores)

In [ ]:
del raw, filt_raw, ica, new_ica